In [59]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re
import nltk
import pprint as pp
import db_scripts
import pprint
import pickle

In [60]:
def get_credentials():
    pkl_file = open('.cred.pkl', 'rb')
    data = pickle.load(pkl_file)
    return data[0], data[1], data[2], date[3]

In [61]:
def load_stop_words(stop_word_file):
    stop_words = []
    fo = open(stop_word_file)
    for line in fo:
        if line.strip()[0:1] != "#":
            for word in line.split():  # in case more than one per line
                stop_words.append(word)
    fo.close()
    return stop_words

In [77]:
def load_keywords_from_bucket(keyword_file):
    word_map = dict()
    file = open(keyword_file)
    for line in file:
        key = line.split(':')
        #print(key)
        x, y = Keywords(key[1]).fetch()
        y = list(set(y))
        if key[0] in word_map:
            val = word_map[key[0]]
            val = [j for i in zip(val,y) for j in i]
            word_map[key[0]] = val
        else:
#             val = list()
#             val.append(y)
            word_map[key[0]] = y
    file.close()
    return word_map
x = load_keywords_from_bucket('keywords.txt')
fp = open('keywords.json','w')
import json
json.dump(x, fp, sort_keys=True, indent=4)
fp.close()

In [63]:
def remove_puncts(text):
    tokenizer = RegexpTokenizer(r'\w+')
    words_no_punct = tokenizer.tokenize(text)
    data = ' '.join(words_no_punct)
    return data

In [64]:
def is_number(s):
    try:
        float(s) if '.' in s else int(s)
        return True
    except ValueError:
        return False

In [65]:
def generate_candidate_keywords(data, stop_words):
    words = [w for w in word_tokenize(
        data) if w.lower() not in stop_words and not is_number(w)]
    return words

In [66]:
def stem_lemmatize(keywords):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    tokenizer = RegexpTokenizer(r'\w+')
    x = []
    y = []
    for word in keywords:
        x.append(stemmer.stem(word))
        y.append(lemmatizer.lemmatize(word))
    return x

In [67]:
def get_percentage_mapping(topic, lemma):
    mapping = [value for value in topic if value in lemma]
    #mapping = [list(filter(lambda x: x in topic, sublist)) for sublist in lemma]
    #print(mapping)
    return (len(mapping)/len(topic))

In [10]:
def assign_topic(cname, domain):
    username, password, db_name, collection_name = get_credentials()
    db = db_scripts.db_connect(username, password, db_name)
    row = db_scripts.db_retrieve(username, password, db_name, collection_name, "CSC", "510")
    #insert a topid attribute in the row and push it back to the database.

In [68]:
class Keywords:
    def __init__(self, text):
        self.text = text
        self.stop_words = load_stop_words("FoxStopList.txt")

    def fetch(self):
        candidate = remove_puncts(self.text)
        keywords = generate_candidate_keywords(candidate, self.stop_words)
        # table = stem_lemmatize(keywords)
        st = stem_lemmatize(keywords)
        return keywords, st
    def map_keywords(self):
        topic_map = load_keywords_from_bucket("keywords.txt")
        stem, lemma = self.fetch()
        percent_mapping = {}
        topic = ''
        maxi = 0
        for key in topic_map.keys():
            percent = get_percentage_mapping(topic_map[key],lemma)
            percent_mapping[key] = percent        
        return percent_mapping

In [69]:
alda = "Introduction to the problems and techniques for automated discovery of knowledge in databases. Topics include representation, evaluation, and formalization of knowledge for discovery; classification, prediction, clustering, and association methods.Selected applications in commerce, security, and bioinformatics. Students cannot get credit for both CSC 422 and CSC 522."
algo = "Algorithm design techniques: use of data structures, divide and conquer, dynamic programming, greedy techniques, local and global search. Complexity and analysis of algorithms: asymptotic analysis, worst case and average case, recurrences, lower bounds, NP-completeness. Algorithms for classical problems including sorting, searching and graph problems (connectivity, shortest paths, minimum spanning trees)."
os = "Fundamental issues related to the design of operating systems. Process scheduling and coordination, deadlock, memory management and elements of distributed systems."
ai = "Introduction to and overview of artificial intelligence. Study of AI programming language such as LISP or PROLOG. Elements of AI problem-solving technique. State spaces and search techniques. Logic, theorem proving and associative databases. Introduction to knowledge representation, expert systems and selected topics including natural language processing, vision and robotics."
ads = "Complex and specialized data structures relevant to design and development of effective and efficient software. Hardware characteristics of storage media. Primary file organizations. Hashing functions and collision resolution techniques. Low level and bit level structures including signatures, superimposed coding, disjoint coding and Bloom filters. Tree and related structures including AVL trees, B*trees, tries and dynamic hashing techniques."

In [57]:
#topic, percent = Keywords(ai).map_keywords()
#topic, percent = Keywords(alda).map_keywords()
#topic, percent = Keywords(os).map_keywords()
# topic, percent = Keywords(algo).map_keywords()
percent = Keywords(ads).map_keywords()

In [58]:
# pprint.pprint(percent)
percent = sorted(percent.items(), key=lambda x:x[1])
pprint.pprint(percent[::-1][:2])

[('Algorithm', 0.25925925925925924), ('Systems', 0.1111111111111111)]


In [26]:
import pandas as pd
df = pd.DataFrame().from_dict(d)

In [27]:
df

,applic,associ,autom,bioinformat,classif,cluster,commerc,databas,discoveri,evalu,formal,knowledg,method,predict,represent,secur
0,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522


In [28]:
d['predict'].append('CSC501')
df = pd.DataFrame().from_dict(d,orient='index').transpose()
df

,evalu,classif,associ,commerc,autom,formal,applic,bioinformat,cluster,databas,discoveri,method,secur,represent,knowledg,predict
0,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522,CSC522
1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,CSC501


In [29]:
import db_scripts
all_courses = db_scripts.db_fetch_all("wolfpal", "courses")

Authentication Successful.


In [30]:
from collections import defaultdict
d = defaultdict()

In [31]:
for course in all_courses:
    l, p = Keywords(course['desc']).fetch()
    # print(course['desc'])
    p = set(p)
    for i in p:
        if i in d:
            d[i].append(course['branch']+course['number'])
        else:
            d[i] = [(course['branch']+course['number'])]

In [33]:
import pandas as pd
df = pd.DataFrame().from_dict(d,orient='index').transpose()

In [34]:
df

,system,element,design,distribut,manag,deadlock,relat,schedul,memori,issu,...,program,prove,techniqu,represent,artifici,databas,intellig,natur,knowledg,space
0,CSC501,CSC501,CSC501,CSC501,CSC501,CSC501,CSC501,CSC501,CSC501,CSC501,...,CSC520,CSC520,CSC520,CSC520,CSC520,CSC520,CSC520,CSC520,CSC520,CSC520
1,CSC520,CSC520,CSC510,None,CSC510,None,CSC541,CSC510,None,None,...,None,None,None,None,None,None,None,None,None,None
2,None,None,CSC541,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [35]:
usersays = "I like AI and systems"
p,l = Keywords(usersays).fetch()
print(l)

['AI', 'system']


In [36]:
for i in l:
    if i in d:
        print("Yeh course le: ",d[i])

Yeh course le:  ['CSC520']
Yeh course le:  ['CSC501', 'CSC520']
